# Scraping Costco Warehouse Location Data

In [1]:
import pandas as pd
import requests, bs4, re
from bs4 import BeautifulSoup as bs
from time import sleep
from selenium import webdriver

In [2]:
d = webdriver.Chrome()
d.get('https://www.costco.com/WarehouseListByStateDisplayView')
sleep(10)
#d.close()

In [44]:
#US states
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}
states=list(states.values())

In [99]:
#scraping location info

storelocations={}

for state in states:
    
    b=d.find_element_by_link_text(state) #https://www.geeksforgeeks.org/click-button-by-text-using-python-and-selenium/
    b.click()
    sleep(2)
    s = bs(d.page_source, 'html.parser').find("div", {"id": "warehouse-by-state-accordion-collapse-panel"})
    stores=s.find_all("div")
    
    for store in stores:
        if store.has_attr('data-warehouse'):
            
            storename=store.find('a', {"class": "warehouse-name h6-style-guide-link"}).text
            statestorename=state+'_'+storename
            storelocations[statestorename]={}
            
            storeaddress=store.find('div', {"class": "h6-style-guide"})
            streetaddress=storeaddress.next.strip()
            city=storeaddress.next.next.next.strip().split(',')[0]
            zipcode=storeaddress.next.next.next.strip().split()[-1]
            storelocations[statestorename]['streetaddress']=streetaddress
            storelocations[statestorename]['city']=city
            storelocations[statestorename]['state']=state
            storelocations[statestorename]['zipcode']=zipcode
            
            storephone=store.find('a', {"class": "body-copy-link"}).text
            storelocations[statestorename]['phonenumber']=storephone.strip()
            
            
            
            icons=store.find_all('i')
            iconlist=[]
            for icon in icons:
                iconlist.append(icon.attrs['class'][0])
            if 'wl-food-court' in iconlist:
                storelocations[statestorename]['food_court']=1
            if 'wl-gas-station' in iconlist:
                storelocations[statestorename]['gas_station']=1
            if 'wl-hearing-aid' in iconlist:
                storelocations[statestorename]['hearing_aid']=1
            if 'wl-optical' in iconlist:
                storelocations[statestorename]['vision_center']=1
            if 'wl-rx' in iconlist:
                storelocations[statestorename]['pharmacy']=1
            if 'wl-tire-center' in iconlist:
                storelocations[statestorename]['tire_center']=1
            if 'wl-business' in iconlist:
                storelocations[statestorename]['business_center']=1


<ipython-input-99-d5bca97ccaac>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  b=d.find_element_by_link_text(state) #https://www.geeksforgeeks.org/click-button-by-text-using-python-and-selenium/


In [101]:
len(storelocations)

607

In [102]:
d.close()

In [103]:
storelocationdf =pd.DataFrame.from_dict(storelocations,orient='index')

In [108]:
storelocationdf.head(5)

,streetaddress,city,state,zipcode,phonenumber,food_court,gas_station,hearing_aid,vision_center,pharmacy,tire_center,business_center
Alaska_Anchorage,330 W DIMOND BLVD,ANCHORAGE,Alaska,99515-1950,(907) 349-2335,1.0,1.0,1.0,1.0,1.0,1.0,NaN
Alaska_Fairbanks,48 COLLEGE RD,FAIRBANKS,Alaska,99701-1706,(907) 205-3607,1.0,1.0,1.0,1.0,1.0,1.0,NaN
Alaska_Juneau,5225 COMMERCIAL BLVD,JUNEAU,Alaska,99801-7210,(907) 780-6740,1.0,NaN,1.0,1.0,1.0,1.0,NaN
Alaska_NE Anchorage,4125 DEBARR RD,ANCHORAGE,Alaska,99508-3115,(907) 269-9510,1.0,1.0,1.0,1.0,1.0,1.0,NaN
Alabama_Hoover,3650 GALLERIA CIR,HOOVER,Alabama,35244-2346,(205) 909-1039,1.0,1.0,1.0,1.0,1.0,1.0,NaN


In [111]:
storelocationdf.to_csv('costcolocations.csv')